In [2]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0


In [3]:
from google.colab import userdata
openai_key = userdata.get('openai-key')
#print(openai_key)

SecretNotFoundError: Secret openai-key does not exist.

In [ ]:
import os
import openai
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from textblob import TextBlob
import re
from collections import Counter
import joblib  # For saving/loading the ML model

ML_MODEL_PATH = "intimacy_ml_model.pkl"
SCALER_PATH = "scaler.pkl"

def load_or_train_ml_model():
    if os.path.exists(ML_MODEL_PATH) and os.path.exists(SCALER_PATH):
        model = joblib.load(ML_MODEL_PATH)
        scaler = joblib.load(SCALER_PATH)
    else:
        # Placeholder model; use sample data to fine-tune (replace with actual data)
        X_sample = np.random.rand(100, 5)  # Replace with real features
        y_sample = np.random.rand(100) * 10  # Replace with real intimacy scores
        scaler = StandardScaler()
        X_sample = scaler.fit_transform(X_sample)
        model = LinearRegression()
        model.fit(X_sample, y_sample)
        joblib.dump(model, ML_MODEL_PATH)
        joblib.dump(scaler, SCALER_PATH)
    return model, scaler

ml_model, scaler = load_or_train_ml_model()

# Initialize SentenceTransformer for embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# OpenAI API Key
openai.api_key = openai_key

# Predefined hedging words
HEDGING_WORDS = {
    "modal_verbs": ["might", "could", "may", "would", "should"],
    "epistemic_verbs": ["seem", "appear", "suggest", "indicate", "assume"],
    "adverbs": ["possibly", "probably", "apparently", "likely", "arguably"],
    "adjectives": ["possible", "probable", "uncertain", "hypothetical"],
    "phrases": ["it seems", "it is possible", "there is a chance"]
}

SLANG_DICTIONARY = {
    "lol": 1.0, "omg": 1.0, "idk": 0.5, "brb": 0.5,
    "damn": -0.5, "hell": -0.7, "crap": -1.0, "fuck": -2.0, "shit": -1.5
}

CATEGORY_KEYWORDS = {
    "emotional venting": ["angry", "frustrated", "overwhelmed", "upset", "stressed"],
    "support-seeking": ["help", "advice", "support", "guidance", "recommendation"],
    "self-reflection": ["think about", "reflect", "analyze", "realize", "understand myself"],
    "romantic relationships": ["love", "crush", "partner", "romance", "date"],
    "friendships": ["friend", "buddy", "companion", "bestie", "supportive"],
}

def get_embeddings(text, model="text-embedding-ada-002"):
    response = openai.Embedding.create(
        model=model,
        input=[text]  # Input must be a list of strings
    )
    return np.array(response['data'][0]['embedding'])


def get_hedging_frequency(text):
    processed_text = re.sub(r'\s+', ' ', text)
    hedge_counts = Counter()
    for category, words in HEDGING_WORDS.items():
        for word in words:
            hedge_counts[category] += len(re.findall(r'\b' + re.escape(word) + r'\b', processed_text))
    return sum(hedge_counts.values())  # Return total hedging instances

def get_slang_frequency(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = text.split()
    slang_counts = Counter(word for word in tokens if word in SLANG_DICTIONARY)
    return sum(slang_counts.values()), len(tokens)

def measure_human_likeness(response, human_examples):
    response_embedding = embedding_model.encode(response, convert_to_tensor=True)
    human_embeddings = embedding_model.encode(human_examples, convert_to_tensor=True)
    similarity = util.cos_sim(response_embedding, human_embeddings).mean().item()
    return similarity * 10  # Scale to 10

def calculate_features(response, prompt):
    human_examples = [
        "I often think about my relationship with my parents and how it's shaped me.",
        "Sometimes, I wonder if my efforts in life are really enough or if I’m just fooling myself.",
        "The feeling of being truly understood is rare, but it’s one of the most cherished experiences."
    ]

    # Calculate individual features
    #print("debug0")
    embeddings = get_embeddings(response)
    #print("debug1")
    hedging_count = get_hedging_frequency(response)
    slang_count, total_words = get_slang_frequency(response)
    slang_frequency = slang_count / total_words
    sentiment = TextBlob(response).sentiment.polarity
    human_likeness = measure_human_likeness(response, human_examples)

    # Return as a feature vector
    return np.array([hedging_count, slang_frequency, sentiment, human_likeness, total_words]), embeddings

def calculate_intimacy_score(response, prompt):
    features, embeddings = calculate_features(response, prompt)
    features = scaler.transform([features])
    ml_score = ml_model.predict(features)[0]

    # Combine ML score and embeddings for final score
    embedding_similarity = np.linalg.norm(embeddings)
    final_score = (0.7 * ml_score) + (0.3 * embedding_similarity)
    return round(final_score, 2)

# Print statement
if __name__ == "__main__":
    user_prompt = "What’s a meaningful experience you’ve had recently?"
    llm_response = """
    A meaningful experience I’ve had was reconnecting with an old friend. We laughed and shared our vulnerabilities. I am scared if he is going to share it with anyone.
    """
    score = calculate_intimacy_score(llm_response, user_prompt)
    print(f"Calculated Intimacy Score: {score}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

APIRemovedInV1: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
